# function

In [3]:
import os

# 查看資料夾內 '同副檔名' 的 '檔名'
def find_filename(folder_path:str=None,     # 資料夾位置
                  File_extension:str=None): # 副檔名
    """  尋找資料夾內 '特定副檔名' 的 '所有檔案的檔名'    """
    filesname = [file for file in os.listdir(folder_path) if file.endswith(f'.{File_extension}')]
    return filesname,len(filesname)


# **縮放圖片**

In [4]:
from PIL import Image

for dataset in ['訓練','驗證','測試']:
    folder_path = f"資料集/貨櫃資料集/{dataset}集/"
    filesname,num = find_filename(folder_path,'jpg')

    for filename in filesname:
        # 打開圖片檔案
        img = Image.open(f"資料集/貨櫃資料集/{dataset}集/{filename}")

        # 使用 resize 函數來縮放圖片，使用 Lanczos 演算法
        resized_img = img.resize((416, 416), Image.Resampling.LANCZOS)

        # 儲存縮放後的圖片為 JPG 檔案
        resized_img.save(f"資料集/貨櫃資料集_縮放/{dataset}集_image/{filename}", "JPEG")

In [8]:
from PIL import Image
# filesname = []
folder_path = f"資料集/圖片準確率測試集/"
filesname,num = find_filename(folder_path,'jpg')
print(filesname)
for filename in filesname:
    img = Image.open(f"{folder_path}{filename}")
    resized_img = img.resize((416, 416), Image.Resampling.LANCZOS)
    resized_img.save(f"資料集/貨櫃資料集_縮放/圖片準確率測試集/{filename}", "JPEG")

['FFAU2895947.jpg', 'MAGU5605323 .jpg', 'SEKU5875349.jpg', 'SEKU5877491.jpg', 'SEKU6026686.jpg', 'TCNU6246126.jpg', 'TLLU4080736.jpg', 'TRHU8927462.jpg', 'TSSU5017340.jpg', 'TSSU5029819.jpg', 'TSSU5042071.jpg', 'TSSU5061615.jpg', 'TSSU5099400.jpg', 'TSSU5142300.jpg', 'TSSU5160351.jpg', 'WHLU5591798.jpg', 'WHLU5842825.jpg', 'WHSU2483178.jpg', 'WHSU2615314.jpg', 'WHSU2864765.jpg', 'WHSU5295430.jpg', 'WHSU5368199.jpg', 'WHSU5563298.jpg', 'WHSU5610492.jpg', 'WHSU5628589.jpg', 'WHSU5744465.jpg', 'WHSU5991104.jpg', 'WHSU5998393.jpg', 'WHSU6010260.jpg', 'WHSU6040178.jpg', 'WHSU6052306.jpg', 'WHSU6167120.jpg', 'WHSU6557387.jpg', 'WHSU6651665.jpg', 'WHSU6856285.jpg']


# **縮放標記**

In [ ]:
import xml.etree.ElementTree as ET

def resize_xml(original_xml_path, new_size, save_xml_path):
    # 載入原始 XML 檔案
    tree = ET.parse(original_xml_path)
    root = tree.getroot()

    # 獲取原始圖像尺寸
    original_width = int(root.find('./size/width').text)
    original_height = int(root.find('./size/height').text)

    # 計算縮放比例
    scale_width = new_size[0] / original_width
    scale_height = new_size[1] / original_height

    # 更新圖像尺寸
    root.find('./size/width').text = str(new_size[0])
    root.find('./size/height').text = str(new_size[1])

    # 更新所有 bndbox 坐標
    for bndbox in root.iter('bndbox'):
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        bndbox.find('xmin').text = str(int(xmin * scale_width))
        bndbox.find('ymin').text = str(int(ymin * scale_height))
        bndbox.find('xmax').text = str(int(xmax * scale_width))
        bndbox.find('ymax').text = str(int(ymax * scale_height))

    # 儲存更新後的 XML 到新的檔案位置
    tree.write(save_xml_path)

In [ ]:
for dataset in ['訓練','驗證','測試']:
    folder_path = f"資料集/貨櫃資料集/{dataset}集_xml/"
    filesname,num = find_filename(folder_path,'xml')

    for filename in filesname:
        original_xml_path = f"資料集/貨櫃資料集/{dataset}集_xml/{filename}"
        save_xml_path = f"資料集/貨櫃資料集_縮放/{dataset}集_xml/{filename}"

        resize_xml(original_xml_path, (416,416), save_xml_path)

# 調整成yolo輸入格式

In [ ]:
import os
import xml.etree.ElementTree as ET

def convert_xml_to_yolo(xml_folder, label_folder, image_size=(416, 416)):
    for xml_file in os.listdir(xml_folder):
        tree = ET.parse(os.path.join(xml_folder, xml_file))
        root = tree.getroot()
        image_width = int(root.find('./size/width').text)
        image_height = int(root.find('./size/height').text)

        # Create a corresponding YOLO label file
        label_file = xml_file.replace('.xml', '.txt')
        with open(os.path.join(label_folder, label_file), 'w') as file:
            for obj in root.iter('object'):
                cls = obj.find('name').text
                cls_id = class_to_id(cls)  # Convert class name to an ID
                xmlbox = obj.find('bndbox')
                x_min = int(xmlbox.find('xmin').text)
                y_min = int(xmlbox.find('ymin').text)
                x_max = int(xmlbox.find('xmax').text)
                y_max = int(xmlbox.find('ymax').text)

                x_center = ((x_min + x_max) / 2) / image_width
                y_center = ((y_min + y_max) / 2) / image_height
                width = (x_max - x_min) / image_width
                height = (y_max - y_min) / image_height

                file.write(f"{cls_id} {x_center} {y_center} {width} {height}\n")

def class_to_id(class_name):
    # This function should map class names to integer IDs
    return {
        'container': 0  # Example: map 'container' to 0
    }.get(class_name, -1)  # Return -1 if class_name is not found

# Example usage
for dataset in ['訓練','驗證','測試']:
    convert_xml_to_yolo(f'資料集/貨櫃資料集_縮放/{dataset}集_xml/', 
                        f'資料集/貨櫃資料集_縮放/{dataset}集_label/')
